## Notebook to compare GOES X-ray flare lists 

There are several ways to access the GOES flare list, and this notebook aims to query them all and inspect the differences.

There ways to access the GOES flare list are:

* NGDC yearly event files from (https://www.ngdc.noaa.gov/stp/space-weather/solar-data/solar-features/solar-flares/x-rays/goes/xrs/) available from 1975-2016.

* Daily SWPC event reports (available to present)

* Query the HEK with the `frm_name==SWPC` attr

* From HELIO (http://voparis-helio.obspm.fr/hec/stfc/HEC_ListsAll.html)

* From the IDL GOES workbench which using `rd_gev.pro`. 

This notebook aims to search over past solar cycle 2010-2018.

In [143]:
import urllib
import pandas as pd 
from sunpy.time import TimeRange
from flarelist_utils import read_swpc_reports, read_ngdc_goes_reports
from dateutil.relativedelta import relativedelta
import pandas as pd 
import numpy as np
import datetime
import glob
import h5netcdf
import matplotlib.pyplot as plt
from scipy.io import readsav
import warnings
warnings.filterwarnings("ignore")

## GEV flare list

Lets look at the flares from the GOES IDL workbench

```
a = ogoes()
gev = a->get_gev('01-Jan-2010', '31-Dec-2018', /struct)
save, gev, filename="goes_flares_from_idl.sav"
```

In [158]:
gev_data = readsav("goes_flares_from_idl.sav")["gev"]
gev_dict = {k : gev_data[k].astype(str) for k in gev_data.dtype.names}
gev_df = pd.DataFrame(gev_dict)

gev_df["goes_class_ind"] = [x[0] for x in gev_df["CLASS"]]
gev_df["datetime"] = pd.to_datetime(gev_df["GSTART"])

gev_df_c = gev_df[gev_df["goes_class_ind"].isin(["C", "M", "X"])]
print(gev_df_c.datetime.min(), gev_df_c.datetime.max())

gev_df_c = gev_df_c.drop_duplicates(subset="datetime")
gev_df_c["match_time"] = gev_df_c.datetime.dt.strftime("%Y-%m-%d %H:%M")

2010-01-02 07:09:00 2018-07-06 19:41:00


## SWPC flare list 

Flare list from daily reports

In [159]:
filedir = "./goes_files/%Y_events/*events*"#%Y%m%devents.txt"
timerange = TimeRange("2010-01-01", "2018-12-31")
years = [timerange.start.datetime + relativedelta(years=i) for i in range(9)]
all_files = []
for y in years:
    all_files += glob.glob(y.strftime(filedir))

all_files.sort()

df_flares = read_swpc_reports(all_files[0])
for f in all_files[1:]:
    df = read_swpc_reports(f)
    df_flares = df_flares.append(df)
df_flares.reset_index(inplace=True, drop=True)
df_flares["ts"] = df_flares.date + df_flares.start_time
df_flares.drop_duplicates(subset="ts")

swpc_c = df_flares[df_flares["goes_class_ind"].isin(["C", "X", "M"])]
swpc_c = swpc_c.drop_duplicates(subset="ts")
swpc_c["datetime"] = pd.to_datetime(swpc_c["date"] + swpc_c["start_time"])
swpc_c["match_time"] = swpc_c.datetime.dt.strftime("%Y-%m-%d %H:%M")

## HEK flares

Flare list by querying the HEK for flares from `a.hek.OBS.Observatory == "GOES"`
This was performed in another notebook.

In [160]:
hek_flares_all = pd.read_csv("solar_cycle24_flares.csv")

In [161]:
hek_flares_all["goes_class_ind"] = [x[0] for x in hek_flares_all["fl_goescls"]]
hek_flares_all["datetime"] = pd.to_datetime(hek_flares_all["event_starttime"])
hek_flares = hek_flares_all[hek_flares_all["goes_class_ind"].isin(["X", "M", "C"])]

hek_flares = hek_flares[(hek_flares.datetime>="2010-01-01")&(hek_flares.datetime<="2018-12-31")]
hek_flares = hek_flares.drop_duplicates(subset="event_starttime")
hek_flares.reset_index(drop=True, inplace=True)

In [162]:
hek_flares_swpc = pd.read_csv("solar_cycle24_flares_swpc.csv")
hek_flares_swpc["datetime"] = pd.to_datetime(hek_flares_swpc["event_starttime"])
hek_flares_swpc["goes_class_ind"] = [x[0] for x in hek_flares_swpc["fl_goescls"]]
hek_flares_swpc["match_time"] = hek_flares_swpc.datetime.dt.strftime("%Y-%m-%d %H:%M")

In [163]:
hek_flares_swpc["ar_noaanum"]

0       11039
1           0
2       11039
3       11039
4       11039
        ...  
8618    12712
8619    12712
8620    12712
8621    12715
8622        0
Name: ar_noaanum, Length: 8623, dtype: int64

## HELIO goes event list

In [164]:
helio_flarelist = pd.read_csv("full_sswlatest.csv")
helio = helio_flarelist[helio_flarelist["goes_class_ind"].isin(["X", "M", "C"])]
helio["datetime"] = pd.to_datetime(helio["time_start"])
helio["match_time"] = ssw.datetime.dt.strftime("%Y-%m-%d %H:%M")
helio = helio.drop_duplicates(subset="match_time")

## Lets print the different numbers of events etc

In [194]:
print_flares(gev_df_c, "gev")
print_flares(swpc_c, "swpc")
print_flares(hek_flares, "hek")
print_flares(helio, "helio")

gev: total: 8522, X: 49, M: 740, C: 7733
swpc: total: 8607, X: 49, M: 748, C: 7810
hek: total: 8619, X: 49, M: 756, C: 7814
helio: total: 8772, X: 49, M: 735, C: 7988


In [195]:
swpc_c["goes_class_ind"].unique()

array(['C', 'M', 'X'], dtype=object)

## Lets match on start time

In [196]:
print(len(swpc_c), len(gev_df_c))
print(len(pd.merge(swpc_c, gev_df_c, on="match_time")))

8607 8522
8517


In [197]:
#common = pd.merge(swpc_c, gev_df_c, on="match_time")
#common = pd.merge(swpc_c, hek_flares_swpc, on="match_time")
common = pd.merge(swpc_c, ssw, on="match_time")
np.all(common["goes_class_ind_x"]== common["goes_class_ind_y"])
common["goes_class_ind"] = common["goes_class_ind_x"]

In [198]:
print_flares(common, "merged")

merged: total: 8232, X: 49, M: 712, C: 7471


In [199]:
len(swpc_c) - len(common)

375

In [200]:
#excess_hek = hek_flares_swpc[~hek_flares_swpc["match_time"].isin(common["match_time"])]
excess_helio = ssw[~ssw["match_time"].isin(common["match_time"])]

In [201]:
len(excess_helio)

541

In [202]:
excess_hek[excess_hek["goes_class_ind"]=="M"]["match_time"]

59      2010-02-08 13:32
67      2010-02-08 21:01
106     2010-06-12 00:30
166     2010-11-04 23:30
247     2011-02-16 01:32
411     2011-03-08 19:46
982     2011-09-23 01:47
1018    2011-09-26 05:06
1662    2012-03-05 22:26
1866    2012-05-05 13:19
2069    2012-06-09 11:20
2071    2012-06-09 16:45
2268    2012-07-06 01:37
2270    2012-07-06 02:44
2508    2012-08-17 13:12
2511    2012-08-17 17:08
2513    2012-08-18 00:24
2514    2012-08-18 03:17
2526    2012-08-18 16:02
2529    2012-08-18 22:46
2530    2012-08-18 23:15
2545    2012-08-30 12:02
2597    2012-09-06 04:06
2611    2012-09-08 17:35
2616    2012-09-09 21:50
3610    2013-07-03 07:00
3937    2013-10-23 23:33
3938    2013-10-23 23:58
3939    2013-10-24 00:21
4078    2013-11-06 23:44
4451    2013-12-31 21:45
4585    2014-01-28 12:38
5855    2014-09-28 02:39
5858    2014-09-28 16:34
5868    2014-10-02 17:10
5869    2014-10-02 18:49
6094    2014-11-07 02:05
6095    2014-11-07 04:12
6694    2015-02-09 22:19
6697    2015-02-09 23:02


In [203]:
swpc_c[~swpc_c["match_time"].isin(common["match_time"])]

,date,event_no,start_time,max_time,end_time,goes_sat,goes_channel,goes_class_ind,goes_class,integrated_flux,noaa_ar,ts,datetime,match_time
107,20100121,5960,0116,0120,0124,G14,1-8A,C,C4.9,1.4E-03,11041,201001210116,2010-01-21 01:16:00,2010-01-21 01:16
168,20100206,6780,2227,2231,2233,G14,1-8A,C,C2.2,4.8E-04,11045,201002062227,2010-02-06 22:27:00,2010-02-06 22:27
169,20100206,6790,2245,2259,2319,G14,1-8A,C,C2.7,4.0E-03,11045,201002062245,2010-02-06 22:45:00,2010-02-06 22:45
172,20100207,6880,0442,0452,0454,G14,1-8A,C,C9.9,2.2E-03,11045,201002070442,2010-02-07 04:42:00,2010-02-07 04:42
188,20100208,7430,0308,0317,0323,G14,1-8A,C,C6.2,3.2E-03,11045,201002080308,2010-02-08 03:08:00,2010-02-08 03:08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14562,20180523,5340,1803,1821,1833,G15,1-8A,C,C2.0,2.4E-03,12712,201805231803,2018-05-23 18:03:00,2018-05-23 18:03
14590,20180528,5680,1704,1710,1712,G15,1-8A,C,C2.7,4.4E-04,12712,201805281704,2018-05-28 17:04:00,2018-05-28 17:04
14624,20180606,6040,1044,1100,1106,G15,1-8A,C,C1.0,6.6E-04,12712,201806061044,2018-06-06 10:44:00,2018-06-06 10:44
14641,20180621,6280,0109,0115,0118,G15,1-8A,C,C2.1,6.4E-04,12715,201806210109,2018-06-21 01:09:00,2018-06-21 01:09


In [204]:
gev_df_c[~gev_df_c["match_time"].isin(common["match_time"])]

,GSTART,GEND,GPEAK,CLASS,LOC,NOAA_AR,goes_class_ind,datetime,match_time
107,21-Jan-2010 01:16:00,21-Jan-2010 01:24:00,21-Jan-2010 01:20:00,C4.9,S24E79,11041,C,2010-01-21 01:16:00,2010-01-21 01:16
150,7-Feb-2010 04:42:00,7-Feb-2010 04:54:00,7-Feb-2010 04:52:00,C9.9,N20E07,11045,C,2010-02-07 04:42:00,2010-02-07 04:42
166,8-Feb-2010 03:08:00,8-Feb-2010 03:23:00,8-Feb-2010 03:17:00,C6.2,N23W00,11045,C,2010-02-08 03:08:00,2010-02-08 03:08
177,8-Feb-2010 13:32:00,8-Feb-2010 13:50:00,8-Feb-2010 13:47:00,M2.0,,11045,M,2010-02-08 13:32:00,2010-02-08 13:32
181,8-Feb-2010 16:33:00,8-Feb-2010 17:12:00,8-Feb-2010 17:07:00,C1.3,N22W09,11045,C,2010-02-08 16:33:00,2010-02-08 16:33
...,...,...,...,...,...,...,...,...,...
14349,23-May-2018 18:03:00,23-May-2018 18:33:00,23-May-2018 18:21:00,C2.0,,12712,C,2018-05-23 18:03:00,2018-05-23 18:03
14377,28-May-2018 17:04:00,28-May-2018 17:12:00,28-May-2018 17:10:00,C2.7,N16E24,12712,C,2018-05-28 17:04:00,2018-05-28 17:04
14411,6-Jun-2018 10:44:00,6-Jun-2018 11:06:00,6-Jun-2018 11:00:00,C1.0,,12712,C,2018-06-06 10:44:00,2018-06-06 10:44
14428,21-Jun-2018 01:09:00,21-Jun-2018 01:18:00,21-Jun-2018 01:15:00,C2.1,N08E15,12715,C,2018-06-21 01:09:00,2018-06-21 01:09


In [205]:
swpc_c[~swpc_c["match_time"].isin(common["match_time"])][["match_time", "goes_class_ind"]]

,match_time,goes_class_ind
107,2010-01-21 01:16,C
168,2010-02-06 22:27,C
169,2010-02-06 22:45,C
172,2010-02-07 04:42,C
188,2010-02-08 03:08,C
...,...,...
14562,2018-05-23 18:03,C
14590,2018-05-28 17:04,C
14624,2018-06-06 10:44,C
14641,2018-06-21 01:09,C


In [206]:
swpc_excess = swpc_c[~swpc_c["match_time"].isin(common["match_time"])][["match_time", "goes_class_ind"]]
swpc_excess[swpc_excess["goes_class_ind"].isin(["M"])]

,match_time,goes_class_ind
199,2010-02-08 13:32,M
211,2010-02-08 21:01,M
524,2010-06-12 00:30,M
1066,2010-11-04 23:30,M
1537,2011-02-16 01:32,M
1749,2011-03-08 19:46,M
2825,2011-09-23 01:47,M
2861,2011-09-26 05:06,M
3731,2012-03-05 22:26,M
4069,2012-05-05 13:19,M


## Final test of swpc reports 2010-2016 and NGDC yearly reports 2010-2016

In [207]:
swpc_c_test = swpc_c[(swpc_c["datetime"]>="2010-01-01")&(swpc_c["datetime"]<="2016-12-31")]

In [177]:
gev_df_test = gev_df_c[(gev_df_c["datetime"]>="2010-01-01")&(gev_df_c["datetime"]<="2016-12-31")]



In [178]:
ngdc_files = glob.glob("./goes_files/*goes-xrs*"); ngdc_files.sort()
ngdc_test = read_ngdc_goes_reports(ngdc_files[0])
for f in ngdc_files[1:]:
    df = read_ngdc_goes_reports(f)
    ngdc_test = ngdc_test.append(df)

ngdc_test.reset_index(inplace=True, drop=True)
ngdc_test["noaa_ar"] = ngdc_test["noaa_ar"].replace(["\n",""],  "     ")

In [179]:
ngdc_test["ts"] = ngdc_test.date + ngdc_test.start_time
ngdc_test.drop_duplicates(subset="ts")

ngdc_test = ngdc_test[ngdc_test["goes_class_ind"].isin(["C", "X", "M"])]
ngdc_test = ngdc_test.drop_duplicates(subset="ts")

In [180]:
print_flares(ngdc_test, "ngdc")
print_flares(swpc_c_test, "swpc")
print_flares(gev_df_test, "gev")

ngdc: X: 45, M: 681, C: 7382
swpc: X: 45, M: 709, C: 7554
gev: X: 45, M: 701, C: 7478


In [181]:
ngdc_test["datetime"] = pd.to_datetime(ngdc_test["date"] + ngdc_test["start_time"])
ngdc_test["match_time"] = ngdc_test.datetime.dt.strftime("%Y-%m-%d %H:%M")

In [182]:
common = pd.merge(swpc_c_test, ngdc_test, on="match_time")
np.all(common["goes_class_ind_x"]== common["goes_class_ind_y"])
common["goes_class_ind"] = common["goes_class_ind_x"]

In [183]:
excess_swpc = swpc_c_test[~swpc_c_test["match_time"].isin(common["match_time"])][["match_time", "goes_class_ind"]]
excess_swpc.head()


,match_time,goes_class_ind
3844,2012-03-22 17:39,C
3850,2012-03-24 08:39,C
3853,2012-03-25 00:15,C
3854,2012-03-25 01:38,C
3855,2012-03-25 08:32,C


In [184]:
excess_ngdc = ngdc_test[~ngdc_test["match_time"].isin(common["match_time"])][["match_time", "goes_class_ind"]]
excess_ngdc.head(18)


,match_time,goes_class_ind
4226,2012-06-07 13:04,C
4541,2012-07-12 07:26,C
4919,2012-09-10 08:26,C
5290,2012-12-03 03:13,C
7508,2013-12-31 16:42,C
7509,2013-12-31 18:20,C
7510,2013-12-31 18:53,C
7511,2013-12-31 19:49,C
7512,2013-12-31 21:45,M
7658,2014-01-28 15:24,M


In [185]:
swpc_c_test[swpc_c_test.datetime>"2014-01-28 00:01"].head(11)

,date,event_no,start_time,max_time,end_time,goes_sat,goes_channel,goes_class_ind,goes_class,integrated_flux,noaa_ar,ts,datetime,match_time
7854,20140128,2210,0031,0036,0039,G15,1-8A,C,C7.6,2.2E-03,11967,201401280031,2014-01-28 00:31:00,2014-01-28 00:31
7855,20140128,2230,0308,0312,0316,G15,1-8A,C,C3.3,1.1E-03,11967,201401280308,2014-01-28 03:08:00,2014-01-28 03:08
7856,20140128,2240,0402,0409,0413,G15,1-8A,M,M1.5,5.6E-03,11967,201401280402,2014-01-28 04:02:00,2014-01-28 04:02
7857,20140128,2260,0525,0529,0531,G15,1-8A,C,C9.2,1.9E-03,11967,201401280525,2014-01-28 05:25:00,2014-01-28 05:25
7858,20140128,2280,0648,0652,0655,G15,1-8A,C,C2.5,7.1E-04,11967,201401280648,2014-01-28 06:48:00,2014-01-28 06:48
7859,20140128,2310,0725,0731,0734,G15,1-8A,M,M3.6,8.0E-03,11967,201401280725,2014-01-28 07:25:00,2014-01-28 07:25
7860,20140128,2350,1134,1138,1141,G15,1-8A,M,M1.4,3.1E-03,11967,201401281134,2014-01-28 11:34:00,2014-01-28 11:34
7861,20140128,2370,1218,1225,1230,G15,1-8A,C,C8.3,3.9E-03,11967,201401281218,2014-01-28 12:18:00,2014-01-28 12:18
7862,20140128,2380,1233,1246,1250,G15,1-8A,M,M1.3,8.9E-03,11967,201401281233,2014-01-28 12:33:00,2014-01-28 12:33
7863,20140128,2400,1623,1645,1657,G15,1-8A,C,C8.5,1.0E-02,11968,201401281623,2014-01-28 16:23:00,2014-01-28 16:23


In [186]:
excess_ngdc[excess_ngdc.goes_class_ind.isin(["M"])]

,match_time,goes_class_ind
7512,2013-12-31 21:45,M
7658,2014-01-28 15:24,M
8396,2014-05-08 09:20,M
10015,2015-02-09 22:19,M
10119,2015-03-07 21:59,M


In [187]:
testy = common[common["noaa_ar_x"]!= common["noaa_ar_y"]][["noaa_ar_x", "noaa_ar_y"]]

In [188]:
for r in testy.values:
    print(r)

['11389' '     ']
['11392' '     ']
['11386' '     ']
['11390' '     ']
['11392' '     ']
['11392' '     ']
['11392' '     ']
['11389' '     ']
['11389' '     ']
['11389' '     ']
['11391' '     ']
['11391' '     ']
['11393' '     ']
['11395' '     ']
['11393' '     ']
['11393' '     ']
['11401' '     ']
['11401' '     ']
['11401' '     ']
['11401' '     ']
['11401' '     ']
['11391' '     ']
['11401' '     ']
['11402' '     ']
['11401' '     ']
['11401' '11403']
['11401' '     ']
['11401' '     ']
['11401' '     ']
['11401' '     ']
['11401' '     ']
['11396' '11402']
['11396' '     ']
['11402' '11401']
['11402' '     ']
['11401' '     ']
['11402' '     ']
['11402' '     ']
['11402' '     ']
['11402' '     ']
['11410' '     ']
['11410' '     ']
['11408' '     ']
['11410' '     ']
['     ' '11410']
['11410' '     ']
['11415' '     ']
['11415' '     ']
['11415' '     ']
['11416' '     ']
['11419' '     ']
['11419' '     ']
['11419' '     ']
['11419' '     ']
['11419' '     ']
['11419' '

['12030' '     ']
['12027' '     ']
['12027' '12030']
['12027' '     ']
['12030' '     ']
['12030' '     ']
['12030' '     ']
['12030' '     ']
['12027' '12021']
['12021' '     ']
['12030' '     ']
['12029' '     ']
['12026' '     ']
['12026' '     ']
['12026' '     ']
['12021' '     ']
['12021' '     ']
['12031' '     ']
['12026' '     ']
['12031' '     ']
['12035' '     ']
['12035' '     ']
['12030' '     ']
['12035' '     ']
['12035' '     ']
['12035' '     ']
['12035' '     ']
['12035' '     ']
['12035' '     ']
['12035' '     ']
['12035' '     ']
['12035' '     ']
['12035' '     ']
['12035' '     ']
['12035' '     ']
['12037' '     ']
['12036' '     ']
['     ' '12036']
['12035' '     ']
['12037' '     ']
['12042' '12037']
['12042' '     ']
['     ' '12035']
['12035' '     ']
['12035' '     ']
['12035' '     ']
['12035' '     ']
['12035' '     ']
['12036' '     ']
['12036' '     ']
['12036' '     ']
['12036' '     ']
['12035' '     ']
['12044' '     ']
['12032' '     ']
['12036' '

['12472' '     ']
['12473' '     ']
['12473' '     ']
['12472' '     ']
['12473' '     ']
['12473' '     ']
['12473' '     ']
['12473' '     ']
['12473' '     ']
['12480' '     ']
['12480' '     ']
['12480' '     ']
['12480' '     ']
['12480' '     ']
['12487' '     ']
['12488' '     ']
['12484' '     ']
['12487' '     ']
['12488' '     ']
['12489' '     ']
['12489' '     ']
['     ' '12486']
['12488' '     ']
['12488' '     ']
['12488' '     ']
['     ' '12491']
['12492' '     ']
['12494' '     ']
['12490' '     ']
['12494' '     ']
['12494' '     ']
['12490' '     ']
['12494' '     ']
['12496' '     ']
['12494' '     ']
['12496' '     ']
['12492' '     ']
['12497' '     ']
['12497' '12496']
['12497' '     ']
['12497' '     ']
['12500' '     ']
['     ' '12497']
['12497' '     ']
['12497' '     ']
['12497' '     ']
['12497' '     ']
['12497' '12500']
['     ' '12497']
['12497' '     ']
['12497' '     ']
['12497' '     ']
['12497' '     ']
['12497' '     ']
['12497' '     ']
['12497' '

In [189]:
hek_flares_swpc["obs_observatory"].unique()

array(['GOES'], dtype=object)

In [193]:
def print_flares(df, name):
    total = len(df)
    x = np.sum(df["goes_class_ind"].isin(["X"]))
    m = np.sum(df["goes_class_ind"].isin(["M"]))
    c = np.sum(df["goes_class_ind"].isin(["C"]))
    print("{:s}: total: {:d}, X: {:d}, M: {:d}, C: {:d}".format(name, total, x, m, c))

In [191]:
swpc_c.head(2)

,date,event_no,start_time,max_time,end_time,goes_sat,goes_channel,goes_class_ind,goes_class,integrated_flux,noaa_ar,ts,datetime,match_time
4,20100102,4300,0709,0724,0740,G14,1-8A,C,C1.0,1.2E-03,11039,201001020709,2010-01-02 07:09:00,2010-01-02 07:09
8,20100102,4370,1408,1416,1424,G14,1-8A,C,C2.6,1.4E-03,,201001021408,2010-01-02 14:08:00,2010-01-02 14:08


In [192]:
gev_df_c.head(2)

,GSTART,GEND,GPEAK,CLASS,LOC,NOAA_AR,goes_class_ind,datetime,match_time
4,2-Jan-2010 07:09:00,2-Jan-2010 07:40:00,2-Jan-2010 07:24:00,C1.0,S29W37,11039,C,2010-01-02 07:09:00,2010-01-02 07:09
8,2-Jan-2010 14:08:00,2-Jan-2010 14:24:00,2-Jan-2010 14:16:00,C2.6,,,C,2010-01-02 14:08:00,2010-01-02 14:08
